In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from nltk import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from nltk.stem import LancasterStemmer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import pickle
from keras import layers, models

In [ ]:
#ASK ZIA ABOUT CREATING DUMMIES/TOKENS FOR TARGET VARIABLE

In [ ]:
train = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
test = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
train.shape, test.shape

In [ ]:
stop_words = stopwords.words('english')
ls = LancasterStemmer()


In [ ]:
train['letter'] = train['context'].apply(lambda x: x[0])
train['anchor_token'] = train['anchor'].apply(lambda x: word_tokenize(x))
train['anchor_len'] = train['anchor_token'].apply(lambda x: len(x))
train['anchor_nostop'] = train['anchor_token'].apply(lambda x: [word for word in x if word not in stop_words])
train['anchor_stem'] = train['anchor_nostop'].apply(lambda x: [ls.stem(word) for word in x])
train['new_anchor'] = train['anchor_stem'].apply(lambda x: ' '.join([str(word) for word in x]))
train['target_token'] = train['target'].apply(lambda x: word_tokenize(x))
train['target_len'] = train['target_token'].apply(lambda x: len(x))
train['target_nostop'] = train['target_token'].apply(lambda x: [word for word in x if word not in stop_words])
train['target_stem'] = train['target_nostop'].apply(lambda x: [ls.stem(word) for word in x])
train['new_target'] = train['target_stem'].apply(lambda x: ' '.join([word for word in x]))
train['final'] = train['new_anchor'] + ' ' + train['new_target']
matchlist = []
for x, y in zip(train['anchor_stem'], train['target_stem']):
  matches = 0
  for word in x:
    if word in y:
      matches += 1
  matchlist.append(matches)
train['matches'] = matchlist
train_dummies = pd.get_dummies(train['letter'])
train = pd.concat([train.drop('letter', axis=1), train_dummies], axis=1)
train['score'] = np.log1p(train['score'])


In [ ]:
test['letter'] = test['context'].apply(lambda x: x[0])
test['anchor_token'] = test['anchor'].apply(lambda x: word_tokenize(x))
test['anchor_len'] = test['anchor_token'].apply(lambda x: len(x))
test['anchor_nostop'] = test['anchor_token'].apply(lambda x: [word for word in x if word not in stop_words])
test['anchor_stem'] = test['anchor_nostop'].apply(lambda x: [ls.stem(word) for word in x])
test['new_anchor'] = test['anchor_stem'].apply(lambda x: ' '.join([str(word) for word in x]))
test['target_token'] = test['target'].apply(lambda x: word_tokenize(x))
test['target_len'] = test['target_token'].apply(lambda x: len(x))
test['target_nostop'] = test['target_token'].apply(lambda x: [word for word in x if word not in stop_words])
test['target_stem'] = test['target_nostop'].apply(lambda x: [ls.stem(word) for word in x])
test['new_target'] = test['target_stem'].apply(lambda x: ' '.join([word for word in x]))
test['final'] = test['new_anchor'] + ' ' + test['new_target']
matchlist = []
for x, y in zip(test['anchor_stem'], test['target_stem']):
  matches = 0
  for word in x:
    if word in y:
      matches += 1
  matchlist.append(matches)
test['matches'] = matchlist
test_dummies = pd.get_dummies(test['letter'])
test = pd.concat([test.drop('letter', axis=1), test_dummies], axis=1)


In [ ]:
vectorizer = CountVectorizer(max_features=None)
vectortrain = vectorizer.fit(train['final'])
pickle.dump(vectortrain, open("patent_vectorizer.pickle", "wb"))
matrixtrain = vectorizer.transform(train['final'])
trainvect = pd.DataFrame(matrixtrain.toarray(), columns=vectortrain.get_feature_names_out())
trainvect['score'] = train.reset_index().score
trainvect['anchor_len'] = train['anchor_len']
trainvect['target_len'] = train['target_len']
trainvect['matches'] = train['matches']
trainvect = pd.concat([trainvect, train_dummies], axis=1)
trainx, testx, trainy, testy = train_test_split(trainvect.drop(['score'], axis=1), trainvect['score'], test_size=.2, random_state=777)

In [ ]:
matrixtrain.shape

In [ ]:
'''
rfc = XGBRegressor()
random_grid = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 6, 10, None],
        'learning_rate':[0.03, 0.06, .1, .15, .2],
        'n_estimators':[100,200,250]}

rf_random = RandomizedSearchCV(estimator=rfc, param_distributions=random_grid,n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(trainx, trainy)

'''


In [ ]:
#rf_random.best_params_

In [ ]:

rfr1 = RandomForestRegressor()
parameters = {
    "n_estimators": [200,300,400],
    "max_features": ["auto", 10, 20],
    "min_samples_split": [6],
    "min_samples_leaf": [1],
    "bootstrap": [True, False],
}


rfr1 = RandomForestRegressor()
parameters = {
    "n_estimators": [200,300,400],
    "max_features": ["auto", 10, 20],
    "min_samples_split": [4,6,8,10],
    "min_samples_leaf": [1,2,3],
    "bootstrap": [True, False],
}


In [ ]:
'''
rfr_grid = GridSearchCV(rfr1,
                        parameters,
                        cv=2,
                        n_jobs=-1,
                        verbose=True)

rfr_grid.fit(trainx,trainy)

print(rfr_grid.best_score_)
print(rfr_grid.best_params_)
'''

In [ ]:
xgb1 = XGBRegressor()
parameters = {'gamma': [0, 1, 2],
              'learning_rate': [.2, 0.25, .1],
              'max_depth': [6, 8, 10, None],
              'min_child_weight': [5],
              'subsample': [1.0],
              'colsample_bytree': [.6],
              'n_estimators': [400]}


xgb1 = RandomForestRegressor()
parameters = {'gamma': [0, 1, 2, 5],
              'learning_rate': [.2, 0.25, .1],
              'max_depth': [4, 6, 8, None],
              'min_child_weight': [4, 5, 6, 7],
              'subsample': [0.7, 0.85, 1.0],
              'colsample_bytree': [0.45, .6, .75],
              'n_estimators': [300, 400, 500]}


In [ ]:
'''
testy = np.log1p(testy)
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv=2,
                        n_jobs=-1,
                        verbose=True)

xgb_grid.fit(trainx, trainy)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)
'''


{'subsample': 1.0,
 'n_estimators': 250,
 'min_child_weight': 5,
 'max_depth': 6,
 'learning_rate': 1,
 'gamma': 1,
 'colsample_bytree': 0.6}

{'n_estimators': 200,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 10,
 'max_depth': None,
 'bootstrap': False}

In [ ]:
testy = np.expm1(testy)

In [ ]:
xgbr = XGBRegressor(subsample= 1.0,n_estimators= 400,min_child_weight= 5,max_depth= 6,learning_rate= .2,gamma= 0,colsample_bytree= 0.6)
xgbr_fit = xgbr.fit(trainx, trainy, eval_set=[(testx, testy)])
pred1 = np.expm1(xgbr_fit.predict(testx))
print(pearsonr(testy, pred1)[0])
pickle.dump(xgbr_fit, open('patent_xgb.pickle', 'wb'))

0.6252846232248347

In [ ]:
rfr = RandomForestRegressor(n_estimators=200, min_samples_leaf=1, min_samples_split=6, max_features=20, bootstrap=True, max_depth=150, max_samples=.6)
rfr_fit = rfr.fit(trainx, trainy)
pred = np.expm1(rfr_fit.predict(testx))
print(pearsonr(testy, pred)[0])
pickle.dump(rfr_fit, open('patent_rfr.pickle', 'wb'))

0.6037753137074942

In [ ]:
pearsonr(testy, (pred1*.5+pred*.5))[0]

In [ ]:
pred

In [ ]:
pred1

In [ ]:
testy

In [ ]:
matrixtest = vectorizer.transform(test['final'])
dftest = pd.DataFrame(matrixtest.toarray(), columns=vectortrain.get_feature_names_out())
dftest['anchor_len'] = test['anchor_len']
dftest['target_len'] = test['target_len']
dftest['matches'] = test['matches']
dftest = pd.concat([dftest, test_dummies], axis=1)
prediction0 = xgbr_fit.predict(dftest)
prediction1 = rfr_fit.predict(dftest)
test['score'] = prediction0*0.5+prediction1*0.5
finaldf = test[['id', 'score']]
#finaldf.to_csv('submission.csv', index=False)

In [ ]:
test['score']

In [ ]:
embedding_dim=16
vocab_size=3000
model = models.Sequential([
  layers.Input(shape=(trainx.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

In [ ]:
model.compile(optimizer='RMSprop',
              loss='mse',
              metrics=['RootMeanSquaredError'])

In [ ]:
model.summary()

In [ ]:
'''
model.fit(
    trainx, trainy,
    validation_data=(testx,testy),
    epochs=41)
pred = model.predict(testx)
pearsonr(np.expm1(testy), np.expm1(pred))
'''

In [ ]:
model.fit(
    np.array(trainvect.drop(['score'], axis=1)), np.array(trainvect['score']),
    epochs=30)
pred = model.predict(np.array(dftest))
modelsub = test[['id']]
modelsub['score'] = np.expm1(pred)
modelsub.to_csv('submission.csv', index=False)

In [ ]:
dftest